In [1]:
import os
import requests
from bs4 import BeautifulSoup
import json 
from dotenv import load_dotenv
from typing import List
from IPython.display import Markdown, display
from openai import OpenAI

In [3]:
load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

if api_key and api_key.startswith('sk-proj-') and len(api_key)>10:
    print("API key wrking fine")
else:
    print("Check api key again")
    
MODEL = 'gpt-4o-mini'
openai = OpenAI()

API key wrking fine


In [4]:
header={
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36'
}

class Website:
    def __init__(self,url):
        self.url=url
        response=requests.get(url, headers=header)
        self.body=response.content
        soup=BeautifulSoup(self.body, 'html.parser')
        self.title=soup.title.string if soup.title else "No title found"
        if soup.body:
            for faaltu in soup.body(['script', 'style','img','input']):
                faaltu.decompose()
            self.text=soup.body.get_text(separator='\n', strip=True)
        else:
            self.text=""
        links=[link.get('href') for link in soup.find_all('a')]
        self.links=[link for link in links if link]

    def get_contents(self):
        return f"Webpage Title:\n{self.title}\nWebpage Contents:\n{self.text}\n\n"

In [5]:
system_prompt="You are provided with a list of links found on a webpage. \
You are able to decide which of the links would be most relevant to include in a brochure about the company, \
such as links to an About page, or a Company page, or Careers/Jobs pages.\n"

system_prompt+='You should respond in JSON as in this example:'
system_prompt+="""
{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page": "url": "https://another.full.url/careers"}
    ]
}
"""

In [6]:
def get_user_prompt(website):
    user_prompt=f'Here is the list of links on the website of {website.url} - '
    user_prompt+="please decide which of these are relevant web links  for a brochure about the company, respond with the full https URL in JSON format. \
        do not include Terms of Service, Privacy, email links.\n"
    user_prompt+="Links (some might be relative links):\nGive the output strictly in python list form"
    user_prompt+="\n".join(website.links)

    return user_prompt

In [7]:
def get_links(url):
    website=Website(url)
    response=openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content":system_prompt},
            {"role":"user", "content": get_user_prompt(website)}
        ],

            response_format={"type":"json_object"}
    )
    result=response.choices[0].message.content
    return json.loads(result)
    

In [8]:
class ConnectTimeout(Exception):
    """Custom exception for connection timeout."""
    pass


In [9]:
import time

def retry(url, retries=5, delay=2):
    for attempt in range(retries):
        try:
            return Website(url).get_contents()  
        except (ConnectTimeout, TimeoutError, Exception) as e:
            print(f"Error occurred for {url}: {e}. Attempt {attempt + 1} of {retries}.")
            time.sleep(delay)  
    print(f"Skipping {url} after {retries} failed attempts.")
    return f"Failed to retrieve contents for {url}\n"

def get_details(url):
    result = "Landing page:\n"
    result += retry(url)  
    links = get_links(url)
    print("Found links:", links)
    
    key_to_check = next(iter(links), None) 
    
    if key_to_check == 'links':
        for link in links["links"]:
            result += f"\n\n{link}\n"
            result += retry(link)  
    elif key_to_check == 'relevant_links':
        for link in links["relevant_links"]:
            result += f"\n\n{link}\n"
            result += retry(link)  
    else:
        result += "\nNo relevant links found.\n"
    
    return result


In [10]:
system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a website ( it can be company website, college club website, someone's portfolio) so be prepared \
and creates a short humorous, entertaining, jokey brochure about the company for prospective customers, investors and recruits. Respond in markdown.\
Include details of company culture, customers and careers/jobs if you have the information."

In [11]:
def brochure_user_prompt(company_name, url):
    user_prompt = f"You are looking at a company called: {company_name}\n"
    user_prompt += f"Here are the contents of its landing page and other relevant pages; use this information to build a professional brief brochure of the website in markdown.At footer always give the contacts link of their social media handles (except phone number) if available\n"
    user_prompt += get_details(url)
    user_prompt = user_prompt[:5_000] 
    return user_prompt

In [12]:
def create_brochure(company_name, url):
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": brochure_user_prompt(company_name, url)}
          ],
    )
    result = response.choices[0].message.content
    display(Markdown(result))

In [13]:
def stream_brochure(company_name, url):
    stream = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": brochure_user_prompt(company_name, url)}
          ],
        stream=True
    )
    
    response = ""
    display_handle = display(Markdown(""), display_id=True)
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        response = response.replace("```","").replace("markdown", "")
        update_display(Markdown(response), display_id=display_handle.display_id)

In [22]:
stream_brochure("Vellore Institute of Technology", "https://www.vit.ac.in")

Found links: {'relevant_links': ['https://viteee.vit.ac.in/?utm_source=VJQY2341&utm_campaign=viteee2025', 'https://vit.ac.in/vit-2025-26-applications-open-for-ug-pg-nri-foreign-research-programmes', 'https://admissions.vit.ac.in/ugapplications/bdes-industrial-design-2025-applications/login', 'https://admissions.vit.ac.in/ugapplications/b-arch-2025-applications/login', 'https://admissions.vit.ac.in/ugapplications/bsc-hons-agri-2025-applications/login', 'https://admissions.vit.ac.in/ugapplications/bsc-bcom-bba-2025-applications/login', 'https://admissions.vit.ac.in/ugnriapplication/login', 'https://results.vit.ac.in/VITREE2025INT', 'https://careers.vit.ac.in/#!/', 'https://admissions.vit.ac.in/researchirapplication/login', 'https://admissions.vit.ac.in/mba-2025-applications/login?utm_source=V71TI1557&utm_campaign=mba2025', 'https://vit.ac.in/about-vit', 'https://vit.ac.in/about/vision-mission', 'https://vit.ac.in/vit-milestones', 'https://vit.ac.in/governance', 'https://vit.ac.in/about/i

# Welcome to Vellore Institute of Technology (VIT)  
## Where Science, Art, and Snacks Meet at the Intersection of Innovation! 

---

### 🎉 **Why VIT? Because Ordinary is Overrated!**  
At VIT, we believe in shaking things up! With a multitude of accolades (212th globally, not to brag…okay, we’re bragging) and a menu of 71 undergraduate and 58 postgraduate programs, we're like the buffet of higher education – there's something for everyone!

---

### 🏆 **Our Superpower: A Top Engineering Institution in India**  
We're not just any university; we’re *India’s Leading University*! Students from all over choose VIT to be trained by the best faculty while pioneering the latest research.  

- **Research?** Why yes, we have 15 research programs!  
- **Degrees?** From B.Des in Industrial Design to Ph.D. in Engineering – take your pick!  
- **Admissions Open:** UG/PG/NRI/Research for 2025-26 – you got options!

---

### 🌍 **Ever Dreamt of Going Global? We’ve Got You!**  
VIT offers an International Transfer Program, inbound exchange opportunities, and a semester abroad experience that will have your passport getting more stamps than your Instagram account!

---

### 🎉 **Campus Life: No Boring Days Allowed!**  
At VIT, you’ll find:

- **Fests:** Because who doesn’t love a good celebration (and free snacks)?
- **Sports:** Channel your inner Olympian, or just enjoy the thrill of dodging the gym!
- **Health Services:** Just in case your excitement goes a bit overboard (party responsibly!).

---

### 🌱 **Sustainable Culture: Saving the Planet, One Campus at a Time!**  
Join us in our *True Green Project*, because we love our campus and want to keep it clean (and green!). Feel free to bring your plant-loving friends along!

---

### 👩‍🎓 **Career Development: Where Jobs Are Made!**  
Worried about finding a job? **Don’t be!** Our Career Development Centre will help you land that dream gig. Did we mention the highest CTC is a whopping ₹1.02 Cr? Cha-ching!

---

### 🌟 **Join Us: Be Part of the VIT Family!**  
Talent is like a diamond in the rough, and at VIT we polish it until it shines! We're searching for faculty and staff who are ready to make a difference – if you're passionate, dedicated, and ready to turn dreams into reality, we want you!

---

### 📢 **Stay Connected!**  
Follow our adventures, announcements, and mischief:  
- [Instagram](#)  
- [Twitter](#)  
- [Facebook](#)  

---

**VIT: A Place to Learn, A Chance to Grow. 🚀**  
Are you ready to take the leap? Apply now and join the fun! 